# Module 02: RDD Basics (Resilient Distributed Datasets)

**Difficulty**: ⭐⭐

**Estimated Time**: 60-75 minutes

**Prerequisites**: 
- Module 00: Introduction to Big Data and Spark Ecosystem
- Module 01: PySpark Setup and SparkSession

## Learning Objectives

By the end of this notebook, you will be able to:
1. Understand what RDDs are and their key properties (immutability, partitioning, fault tolerance)
2. Create RDDs from Python collections and external files
3. Distinguish between transformations and actions
4. Explain lazy evaluation and how Spark optimizes computation
5. Work with key-value pair RDDs for advanced operations

## Setup

In [ ]:
# Import required libraries
from pyspark.sql import SparkSession
import time

# Create SparkSession
spark = SparkSession.builder \
    .appName("Module 02: RDD Basics") \
    .master("local[*]") \
    .config("spark.driver.memory", "2g") \
    .getOrCreate()

# Access SparkContext (needed for RDD operations)
sc = spark.sparkContext

print(f"✓ SparkSession created: {spark.sparkContext.appName}")
print(f"  Spark UI: {spark.sparkContext.uiWebUrl}")
print(f"  Using {sc.defaultParallelism} CPU cores")

## 1. What is an RDD?

### Definition

**RDD (Resilient Distributed Dataset)** is Spark's fundamental data abstraction:

- **Resilient**: Fault-tolerant, can recover from node failures
- **Distributed**: Data is partitioned across multiple machines
- **Dataset**: Collection of objects (like a Python list)

### Key Properties

```
┌─────────────────────────────────────┐
│            RDD Properties           │
├─────────────────────────────────────┤
│ 1. Immutable   → Cannot be changed  │
│ 2. Distributed → Split into parts   │
│ 3. Partitioned → Parallel processing│
│ 4. Lazy        → Computed on demand │
│ 5. Typed       → Contains objects   │
└─────────────────────────────────────┘
```

### RDD vs Python List

| Feature | Python List | RDD |
|---------|-------------|-----|
| **Location** | Single machine (in memory) | Distributed across cluster |
| **Size limit** | Machine's RAM | Virtually unlimited |
| **Processing** | Sequential | Parallel |
| **Mutability** | Mutable | Immutable |
| **Fault tolerance** | None | Automatic recovery |

### When to Use RDDs?

**Modern Spark (prefer DataFrames for most tasks)**:
- DataFrames are optimized and easier to use
- Use RDDs only when you need:
  - Low-level control over data partitioning
  - Custom partitioners
  - Unstructured data (text files, logs)
  - Working with existing RDD-based codebases

## 2. Creating RDDs

### Method 1: Parallelizing a Collection

In [ ]:
# Create RDD from Python list
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
numbers_rdd = sc.parallelize(numbers)

print(f"RDD created: {numbers_rdd}")
print(f"Type: {type(numbers_rdd)}")
print(f"Number of partitions: {numbers_rdd.getNumPartitions()}")

# View the data (this is an ACTION - triggers computation)
print(f"\nFirst 5 elements: {numbers_rdd.take(5)}")
print(f"All elements: {numbers_rdd.collect()}")

### Controlling Partitions

In [ ]:
# Create RDD with specific number of partitions
numbers_4_partitions = sc.parallelize(numbers, numSlices=4)
numbers_2_partitions = sc.parallelize(numbers, numSlices=2)

print(f"Default partitions: {numbers_rdd.getNumPartitions()}")
print(f"With 4 partitions: {numbers_4_partitions.getNumPartitions()}")
print(f"With 2 partitions: {numbers_2_partitions.getNumPartitions()}")

# View how data is distributed across partitions
print("\nData distribution with 4 partitions:")
partitions = numbers_4_partitions.glom().collect()
for i, partition in enumerate(partitions):
    print(f"  Partition {i}: {list(partition)}")

### Method 2: Reading from External Files

In [ ]:
# Create a sample text file first
sample_text = """Apache Spark is a unified analytics engine
For large-scale data processing
Spark provides high-level APIs in Java, Scala, Python and R
It also supports a rich set of higher-level tools
Including Spark SQL for SQL and structured data processing"""

# Save to file
with open('sample_text.txt', 'w') as f:
    f.write(sample_text)

# Create RDD from text file
text_rdd = sc.textFile('sample_text.txt')

print(f"Text RDD: {text_rdd}")
print(f"Number of lines: {text_rdd.count()}")
print(f"\nFirst 3 lines:")
for line in text_rdd.take(3):
    print(f"  {line}")

## 3. Transformations vs Actions

This is one of the most important concepts in Spark!

### Transformations (Lazy)

**Transformations** create a new RDD from an existing one:
- **Lazy**: Not computed immediately
- **Return**: New RDD
- **Examples**: `map()`, `filter()`, `flatMap()`, `reduceByKey()`

### Actions (Eager)

**Actions** trigger computation and return results:
- **Eager**: Execute immediately
- **Return**: Values to driver program
- **Examples**: `collect()`, `count()`, `take()`, `reduce()`

### Visual Representation

```
RDD1 → map() → RDD2 → filter() → RDD3 → collect()
       ↑                ↑                  ↑
   Transformation   Transformation      Action
   (not executed)   (not executed)   (executes all!)
```

### Common Transformations

In [ ]:
# Create sample RDD
numbers = sc.parallelize([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

# 1. map() - Apply function to each element
squared = numbers.map(lambda x: x ** 2)
print(f"map() - Square each number: {squared.collect()}")

# 2. filter() - Keep elements matching condition
evens = numbers.filter(lambda x: x % 2 == 0)
print(f"filter() - Even numbers only: {evens.collect()}")

# 3. flatMap() - Map then flatten
words = sc.parallelize(["Hello World", "Apache Spark"])
flat_words = words.flatMap(lambda x: x.split(" "))
print(f"flatMap() - Split into words: {flat_words.collect()}")

# 4. distinct() - Remove duplicates
duplicates = sc.parallelize([1, 2, 2, 3, 3, 3, 4])
unique = duplicates.distinct()
print(f"distinct() - Remove duplicates: {unique.collect()}")

# 5. union() - Combine two RDDs
rdd1 = sc.parallelize([1, 2, 3])
rdd2 = sc.parallelize([4, 5, 6])
combined = rdd1.union(rdd2)
print(f"union() - Combine RDDs: {combined.collect()}")

### Common Actions

In [ ]:
numbers = sc.parallelize([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

# 1. collect() - Return all elements (be careful with large datasets!)
all_nums = numbers.collect()
print(f"collect() - All elements: {all_nums}")

# 2. count() - Count number of elements
total = numbers.count()
print(f"count() - Total elements: {total}")

# 3. first() - Get first element
first_num = numbers.first()
print(f"first() - First element: {first_num}")

# 4. take(n) - Get first n elements
first_three = numbers.take(3)
print(f"take(3) - First 3 elements: {first_three}")

# 5. reduce() - Aggregate using function
sum_all = numbers.reduce(lambda a, b: a + b)
print(f"reduce() - Sum of all: {sum_all}")

# 6. foreach() - Apply function to each element (no return)
# Useful for side effects like printing or saving
print("foreach() - Print each element:")
numbers.take(3).foreach(print)  # Using take to limit output

## 4. Lazy Evaluation

### Understanding Lazy Evaluation

Spark doesn't execute transformations immediately. Instead:

1. **Builds a DAG** (Directed Acyclic Graph) of operations
2. **Optimizes** the computation plan
3. **Executes** only when an action is called

### Demonstration

In [ ]:
# Create RDD
print("Step 1: Creating RDD...")
data = sc.parallelize(range(1, 1001))  # 1000 numbers
print("  RDD created (not computed yet!)\n")

# Apply transformations (these are LAZY)
print("Step 2: Applying transformations...")
squared = data.map(lambda x: x ** 2)
print("  Transformation 1: map (not executed!)")

filtered = squared.filter(lambda x: x > 100000)
print("  Transformation 2: filter (not executed!)")

sorted_rdd = filtered.sortBy(lambda x: x)
print("  Transformation 3: sortBy (not executed!)\n")

# Trigger action (this executes ALL transformations)
print("Step 3: Calling action (collect)...")
start_time = time.time()
result = sorted_rdd.take(5)  # Action!
end_time = time.time()

print(f"  Action completed in {end_time - start_time:.4f} seconds")
print(f"  Result: {result}")
print("\n✓ All transformations executed when action was called!")

### Benefits of Lazy Evaluation

1. **Optimization**: Spark can optimize the entire pipeline before execution
2. **Efficiency**: Avoids computing intermediate results that aren't needed
3. **Fault tolerance**: Can recreate lost partitions by replaying transformations

### Viewing the DAG

You can visualize the execution plan in the Spark UI:
1. Open Spark UI (check URL printed at the beginning)
2. Go to "DAG Visualization" tab
3. See how Spark optimized your transformations

In [ ]:
# Print lineage (shows transformation chain)
print("RDD Lineage (transformation chain):")
print(sorted_rdd.toDebugString().decode('utf-8'))

## 5. Key-Value Pair RDDs

### Creating Pair RDDs

Pair RDDs are RDDs of tuples: `(key, value)`

They enable powerful operations like:
- `reduceByKey()`: Aggregate by key
- `groupByKey()`: Group by key
- `sortByKey()`: Sort by key
- `join()`: Join two RDDs by key

In [ ]:
# Create pair RDD from regular RDD
fruits = sc.parallelize(["apple", "banana", "apple", "orange", "banana", "apple"])

# Convert to (fruit, 1) pairs for counting
fruit_pairs = fruits.map(lambda x: (x, 1))

print("Pair RDD:")
print(fruit_pairs.collect())

### reduceByKey() - Aggregate by Key

In [ ]:
# Count occurrences of each fruit
fruit_counts = fruit_pairs.reduceByKey(lambda a, b: a + b)

print("Fruit counts (using reduceByKey):")
print(fruit_counts.collect())

# More examples with numbers
sales = sc.parallelize([
    ("Product A", 100),
    ("Product B", 200),
    ("Product A", 150),
    ("Product C", 300),
    ("Product B", 50)
])

total_sales = sales.reduceByKey(lambda a, b: a + b)
print("\nTotal sales by product:")
for product, total in total_sales.collect():
    print(f"  {product}: ${total}")

### groupByKey() vs reduceByKey()

In [ ]:
# groupByKey() - Groups all values for each key
sales_grouped = sales.groupByKey()

print("Grouped sales (using groupByKey):")
for product, values in sales_grouped.collect():
    print(f"  {product}: {list(values)}")

# ⚠️ Important: reduceByKey() is usually more efficient!
# - reduceByKey: Reduces locally before shuffling (less data transfer)
# - groupByKey: Shuffles all data (can cause memory issues)

print("\n💡 Tip: Prefer reduceByKey() over groupByKey() for aggregations")

### Other Pair RDD Operations

In [ ]:
# sortByKey() - Sort by key
sorted_sales = total_sales.sortByKey()
print("Sorted by product name:")
print(sorted_sales.collect())

# mapValues() - Apply function to values only (keys unchanged)
sales_with_tax = total_sales.mapValues(lambda x: x * 1.10)  # Add 10% tax
print("\nSales with 10% tax:")
for product, total in sales_with_tax.collect():
    print(f"  {product}: ${total:.2f}")

# keys() and values() - Extract keys or values
products = total_sales.keys().collect()
amounts = total_sales.values().collect()
print(f"\nProducts: {products}")
print(f"Amounts: {amounts}")

## Exercises

### Exercise 1: Word Count (Classic Spark Example)

Implement the classic word count program:
1. Read the text file created earlier
2. Split each line into words
3. Count occurrences of each word
4. Sort by count (descending)

In [ ]:
# Exercise 1: Your code here

# Step 1: Read text file
text = sc.textFile('sample_text.txt')

# Step 2: Split into words (use flatMap)
# Your code here

# Step 3: Create (word, 1) pairs
# Your code here

# Step 4: Count by word
# Your code here

# Step 5: Sort by count (descending)
# Your code here

# Print top 5 words
# Your code here

### Exercise 2: Filter and Transform

Given a list of temperatures in Celsius:
1. Create an RDD
2. Filter temperatures above 25°C
3. Convert to Fahrenheit: `F = C * 9/5 + 32`
4. Count how many hot days (>77°F)

In [ ]:
# Exercise 2: Your code here

temperatures_c = [22, 28, 31, 19, 26, 33, 18, 29, 25, 30]

# Create RDD
# Your code here

# Filter > 25°C
# Your code here

# Convert to Fahrenheit
# Your code here

# Count > 77°F
# Your code here

### Exercise 3: Sales Analysis

Analyze sales data:
```python
sales_data = [
    ("Electronics", 500),
    ("Clothing", 200),
    ("Electronics", 700),
    ("Food", 150),
    ("Clothing", 300),
    ("Electronics", 400),
    ("Food", 250)
]
```

Tasks:
1. Calculate total sales per category
2. Find the category with highest sales
3. Calculate average sales per category

In [ ]:
# Exercise 3: Your code here

sales_data = [
    ("Electronics", 500),
    ("Clothing", 200),
    ("Electronics", 700),
    ("Food", 150),
    ("Clothing", 300),
    ("Electronics", 400),
    ("Food", 250)
]

# Create RDD
# Your code here

# Task 1: Total sales per category
# Your code here

# Task 2: Category with highest sales
# Your code here

# Task 3: Average sales per category
# Hint: Use mapValues with groupByKey or aggregate functions
# Your code here

### Exercise 4: Understanding Lazy Evaluation

Predict the output:
```python
rdd1 = sc.parallelize([1, 2, 3, 4])
rdd2 = rdd1.map(lambda x: x * 2)
rdd3 = rdd2.filter(lambda x: x > 4)
print("Done with transformations")
result = rdd3.collect()
print(result)
```

Questions:
1. At which point does Spark actually compute the values?
2. What would `rdd2.collect()` return?
3. What would `rdd3.collect()` return?

In [ ]:
# Exercise 4: Test your predictions

rdd1 = sc.parallelize([1, 2, 3, 4])
rdd2 = rdd1.map(lambda x: x * 2)
rdd3 = rdd2.filter(lambda x: x > 4)
print("Done with transformations")

# Uncomment to test:
# print(f"rdd2: {rdd2.collect()}")
# print(f"rdd3: {rdd3.collect()}")

## Solutions

### Exercise 1 Solution

In [ ]:
# Solution 1: Word Count

text = sc.textFile('sample_text.txt')
words = text.flatMap(lambda line: line.split())
word_pairs = words.map(lambda word: (word, 1))
word_counts = word_pairs.reduceByKey(lambda a, b: a + b)
sorted_counts = word_counts.sortBy(lambda x: x[1], ascending=False)

print("Top 10 words:")
for word, count in sorted_counts.take(10):
    print(f"  {word}: {count}")

### Exercise 2 Solution

In [ ]:
# Solution 2: Temperature Analysis

temperatures_c = [22, 28, 31, 19, 26, 33, 18, 29, 25, 30]

temps_rdd = sc.parallelize(temperatures_c)
hot_days_c = temps_rdd.filter(lambda t: t > 25)
hot_days_f = hot_days_c.map(lambda c: c * 9/5 + 32)
very_hot_count = hot_days_f.filter(lambda f: f > 77).count()

print(f"Hot days in Celsius (>25°C): {hot_days_c.collect()}")
print(f"Hot days in Fahrenheit: {hot_days_f.collect()}")
print(f"Number of very hot days (>77°F): {very_hot_count}")

### Exercise 3 Solution

In [ ]:
# Solution 3: Sales Analysis

sales_data = [
    ("Electronics", 500), ("Clothing", 200), ("Electronics", 700),
    ("Food", 150), ("Clothing", 300), ("Electronics", 400), ("Food", 250)
]

sales_rdd = sc.parallelize(sales_data)

# Task 1: Total sales per category
total_by_category = sales_rdd.reduceByKey(lambda a, b: a + b)
print("Total sales by category:")
for category, total in total_by_category.collect():
    print(f"  {category}: ${total}")

# Task 2: Highest sales category
highest = total_by_category.sortBy(lambda x: x[1], ascending=False).first()
print(f"\nHighest sales: {highest[0]} (${highest[1]})")

# Task 3: Average sales per category
count_by_category = sales_rdd.mapValues(lambda x: (x, 1)) \
    .reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1]))
avg_by_category = count_by_category.mapValues(lambda x: x[0] / x[1])

print("\nAverage sales by category:")
for category, avg in avg_by_category.collect():
    print(f"  {category}: ${avg:.2f}")

### Exercise 4 Solution

**Answers**:
1. Spark computes values when `collect()` is called (an action)
2. `rdd2.collect()` would return `[2, 4, 6, 8]`
3. `rdd3.collect()` would return `[6, 8]` (values > 4)

The transformations are not executed until an action is called!

## Summary

### Key Concepts Covered

✅ **RDD Properties**: Resilient, Distributed, Immutable, Partitioned, Lazy

✅ **Creating RDDs**: From collections (`parallelize`) and files (`textFile`)

✅ **Transformations**: Lazy operations that create new RDDs (map, filter, flatMap)

✅ **Actions**: Eager operations that return results (collect, count, reduce)

✅ **Lazy Evaluation**: Transformations are not computed until an action is called

✅ **Pair RDDs**: Key-value operations (reduceByKey, groupByKey, sortByKey)

### Important Transformations and Actions

**Transformations** (return RDD):
- `map()`, `flatMap()`, `filter()`, `distinct()`
- `union()`, `intersection()`, `subtract()`
- `reduceByKey()`, `groupByKey()`, `sortByKey()`

**Actions** (return values):
- `collect()`, `count()`, `first()`, `take(n)`
- `reduce()`, `foreach()`, `saveAsTextFile()`

### Best Practices

1. **Avoid `collect()` on large RDDs**: It brings all data to driver (memory issues)
2. **Use `reduceByKey()` over `groupByKey()`**: More efficient, less shuffle
3. **Control partitions**: More partitions = more parallelism (but overhead)
4. **Understand lazy evaluation**: Helps with debugging and optimization
5. **Prefer DataFrames**: In modern Spark, use DataFrames for structured data

### What's Next?

In **Module 03: DataFrames and Datasets**, you will:
- Learn the DataFrame API (higher-level abstraction)
- Understand schema and structured data
- Convert between RDDs and DataFrames
- Use Spark's Catalyst optimizer

### Additional Resources

- [RDD Programming Guide](https://spark.apache.org/docs/latest/rdd-programming-guide.html)
- [PySpark RDD API](https://spark.apache.org/docs/latest/api/python/reference/pyspark.html#rdd-apis)

In [ ]:
# Cleanup
spark.stop()
print("SparkSession stopped. ✓")